In [1]:
!wget https://download.pytorch.org/tutorial/data.zip

--2023-02-09 22:27:28--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 99.84.192.8, 99.84.192.10, 99.84.192.129, ...
Connecting to download.pytorch.org (download.pytorch.org)|99.84.192.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  6.51MB/s    in 0.4s    

2023-02-09 22:27:30 (6.51 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [90]:
import torch 
import torch.nn as nn
import os
import unicodedata

In [110]:
lett = set()
g = " .,;'"
for i in g:
  lett.add(i)
lett

{' ', "'", ',', '.', ';'}

In [111]:
cat = {}
for i in os.listdir("data/names"):
  with open(os.path.join("data","names",i),"r") as f:
    nd = f.read().split("\n")
    for j in nd:
      for k in j:
        k = unicodedata.normalize('NFKD',k).encode('ascii', 'ignore').decode()
        lett.add(k)
    cat[i[:-4]] = nd
n_cat = len(cat)

In [119]:
ltoi = {j:i for i,j in enumerate(lett)}
ln_l = len(ltoi)
print(ltoi)
ln_l

{'': 0, ',': 1, 'D': 2, 'M': 3, 'A': 4, 'E': 5, 'X': 6, 't': 7, 'd': 8, 'm': 9, 'j': 10, '/': 11, 'q': 12, '-': 13, 'n': 14, 'g': 15, 'V': 16, ':': 17, 'Q': 18, 'k': 19, 'v': 20, ' ': 21, 'L': 22, 'N': 23, "'": 24, 'c': 25, '1': 26, 'p': 27, 'z': 28, 'w': 29, 'Z': 30, 's': 31, 'e': 32, 'T': 33, 'h': 34, 'O': 35, 'y': 36, 'S': 37, 'G': 38, 'u': 39, 'C': 40, 'o': 41, 'f': 42, 'r': 43, 'R': 44, 'P': 45, 'H': 46, 'B': 47, 'U': 48, 'x': 49, '.': 50, 'i': 51, 'J': 52, 'a': 53, 'I': 54, 'W': 55, 'F': 56, ';': 57, 'Y': 58, 'K': 59, 'b': 60, 'l': 61}


62

In [120]:
print(cat['Italian'][:5])
n_cat

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


18

In [168]:
def enc(letter):
  tempt = torch.tensor([])
  for idx,ele in enumerate(letter):
    c = torch.nn.functional.one_hot(torch.tensor(ltoi[ele]),num_classes=ln_l)
    tempt = torch.cat((tempt,c.unsqueeze(0)),dim=0)
  return tempt.view(-1,1,62)

In [169]:
f = enc("Jones")

In [174]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(62, n_hidden, n_cat)

In [175]:
rnn

RNN(
  (i2h): Linear(in_features=190, out_features=128, bias=True)
  (i2o): Linear(in_features=190, out_features=18, bias=True)
  (softmax): LogSoftmax(dim=1)
)